In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import os
import cv2
import seaborn as sns
import matplotlib.pyplot as plt
import gc
import pickle
from plotting import loss_accuracy_graph_from_pickle, loss_accuracy_graph, bar_accuracy_plot, scatter_plot

import keras
from keras.preprocessing.image import ImageDataGenerator, DirectoryIterator
from keras.utils import to_categorical
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from vit_keras import vit

from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix

import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.backend import clear_session
from tensorflow.keras.preprocessing.image import ImageDataGenerator

C:\Users\User\anaconda3\envs\GPU3\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [2]:
data_path = r'C:\Users\User\Desktop\Master_Y1\Deep_learning\BrainTumorDetection-main\brain_tumor_dataset'
classes = ['no', 'yes']
def create_inception_v3():
    base_incep = InceptionV3(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))
    x = base_incep.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(len(classes), activation='softmax')(x)
    inception = Model(inputs=base_incep.input, outputs=predictions)
    for layer in base_incep.layers:
        layer.trainable = False
    inception.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy'])
    return inception

def create_resnet50():
    base_resn = ResNet50(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))
    x = base_resn.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(len(classes), activation='softmax')(x)
    resnet = Model(inputs=base_resn.input, outputs=predictions)
    for layer in base_resn.layers:
        layer.trainable = False
    resnet.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy'])
    return resnet

def create_vgg16():
    base_model = keras.applications.VGG16(weights='imagenet', include_top=False) 
    base_model.trainable = False
    inputs = keras.Input((224, 224, 3))
    x = base_model(inputs, training=False)
    x = keras.layers.GlobalAveragePooling2D()(x)
    outputs = keras.layers.Dense(2, activation='sigmoid')(x)
    VGG = keras.Model(inputs, outputs, name="VGG")
    VGG.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy'])
    return VGG

def create_vit_b16():
    b16 = vit.vit_b16(image_size=(img_size, img_size), pretrained=True, activation='sigmoid', include_top=True, pretrained_top=False, classes=2)
    b16.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy'])
    return b16

def create_vit_b32():
    b32 = vit.vit_b32(image_size=(img_size, img_size), pretrained=True, activation='sigmoid', include_top=True, pretrained_top=False, classes=2)
    b32.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy'])
    return b32

def create_vit_l16():
    l16 = vit.vit_l16(image_size=(224,224), activation='sigmoid', include_top=True, pretrained_top=False, classes=2)
    l16.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy'])
    return l16


In [3]:
def load_and_preprocess_data(img_size):
    data_path = r'C:\Users\User\Desktop\Master_Y1\Deep_learning\BrainTumorDetection-main\brain_tumor_dataset'
    classes = ['no', 'yes']

    X = []
    Y = []
    for c in classes:
        path = os.path.join(data_path, c)
        class_num = classes.index(c)
        for img in os.listdir(path):
            img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_COLOR)
            img_arr = cv2.resize(img_arr, (img_size, img_size))
            X.append(img_arr)
            Y.append(class_num)

    X = np.array(X)
    Y = np.array(Y)
    Y = to_categorical(Y, num_classes=len(classes))

    kf = KFold(n_splits=5, shuffle=True, random_state=42)

    datagen = ImageDataGenerator(
    rotation_range=360,
    horizontal_flip=True)

    return X, Y, kf, datagen

In [4]:
img_size = 224
epochs = 100
batch_size = 16
data_dictionary = {}

X, Y, kf, datagen = load_and_preprocess_data(img_size)

In [5]:
def cross_validate(create_model_fn, X, Y, kf, datagen, batch_size=8, epochs=10, num_augmented_images_per_original=2):

    entire_history = []
    best_history = None
    best_val_accuracy = -np.inf

    for train_index, val_index in kf.split(X):
        X_train, X_val = X[train_index], X[val_index]
        Y_train, Y_val = Y[train_index], Y[val_index]

        # Generate augmented data
        augmented_X_train = []
        augmented_Y_train = []

        for x, y in zip(X_train, Y_train):
            x = np.expand_dims(x, axis=0)
            y = np.expand_dims(y, axis=0)

            img_gen = datagen.flow(x, y, batch_size=1)

            for i in range(num_augmented_images_per_original):
                aug_x, aug_y = next(img_gen)
                augmented_X_train.append(aug_x[0])
                augmented_Y_train.append(aug_y[0])

        augmented_X_train = np.array(augmented_X_train)
        augmented_Y_train = np.array(augmented_Y_train)

        # Combine original and augmented training data
        combined_X_train = np.concatenate((X_train, augmented_X_train), axis=0)
        combined_Y_train = np.concatenate((Y_train, augmented_Y_train), axis=0)

        clear_session()

        # Create a new instance of the model with reset weights
        model = create_model_fn()

        train_generator = datagen.flow(combined_X_train, combined_Y_train, batch_size=batch_size)

        model_history = model.fit(train_generator, steps_per_epoch=len(combined_X_train) // batch_size, epochs=epochs, validation_data=(X_val, Y_val))

        entire_history.append(model_history.history)
        current_val_accuracy = model_history.history['val_binary_accuracy'][-1]

        del model
        del model_history
        del X_train
        del X_val
        del Y_train
        del Y_val
        del combined_X_train
        del combined_Y_train
        del augmented_X_train
        del augmented_Y_train
        gc.collect()


    return best_history ,best_model, best_val_x, best_val_y, entire_history

In [ ]:
inception_history, inception_model, inception_val_x, inception_val_y, entire_inception_history = cross_validate(create_inception_v3, X, Y, kf, datagen, batch_size=batch_size, epochs=epochs)

In [ ]:
data_dictionary["inception"] = [inception_val_x, inception_val_y]

file_path = "inception_history.pickle"

with open(file_path, 'wb') as file_pi:
    pickle.dump(entire_inception_history, file_pi)

In [ ]:
inception_model.save('inception_model.h5')

In [ ]:
resnet_history, resnet_model, resnet_val_x, resnet_val_y, entire_resnet_history = cross_validate(create_resnet50, X, Y, kf, datagen, batch_size=batch_size, epochs=epochs)

In [ ]:
data_dictionary["resnet"] = [resnet_val_x, resnet_val_y]

file_path = "resnet_history.pickle"

with open(file_path, 'wb') as file_pi:
    pickle.dump(entire_resnet_history, file_pi)

In [ ]:
resnet_model.save('resnet_model.h5')

In [ ]:
vgg16_history, vgg16_model, vgg16_val_x, vgg16_val_y, entire_vgg16_history = cross_validate(create_vgg16, X, Y, kf, datagen, batch_size=batch_size, epochs=epochs)

In [ ]:
data_dictionary["vgg16"] = [vgg16_val_x, vgg16_val_y]

file_path = "vgg16_history.pickle"

with open(file_path, 'wb') as file_pi:
    pickle.dump(entire_vgg16_history, file_pi)

In [ ]:
vgg16_model.save('vgg16_model.h5')

In [6]:
b16_history, b16_model, b16_val_x, b16_val_y, entire_b16_history = cross_validate(create_vit_b16, X, Y, kf, datagen, batch_size=batch_size, epochs=epochs)

C:\Users\User\anaconda3\envs\GPU3\lib\site-packages\vit_keras\utils.py:81: UserWarning: Resizing position embeddings from 24, 24 to 14, 14
  warnings.warn(


Epoch 1/100
37/37 [==============================] - 51s 755ms/step - loss: 0.8031 - binary_accuracy: 0.5915 - val_loss: 0.6609 - val_binary_accuracy: 0.6078
Epoch 2/100
37/37 [==============================] - 25s 662ms/step - loss: 0.6703 - binary_accuracy: 0.6237 - val_loss: 0.6614 - val_binary_accuracy: 0.5784
Epoch 3/100
37/37 [==============================] - 24s 657ms/step - loss: 0.6470 - binary_accuracy: 0.6203 - val_loss: 0.5969 - val_binary_accuracy: 0.6471
Epoch 4/100
37/37 [==============================] - 24s 647ms/step - loss: 0.6347 - binary_accuracy: 0.6661 - val_loss: 0.5667 - val_binary_accuracy: 0.7647
Epoch 5/100
37/37 [==============================] - 24s 650ms/step - loss: 0.5990 - binary_accuracy: 0.6983 - val_loss: 0.5371 - val_binary_accuracy: 0.7647
Epoch 6/100
37/37 [==============================] - 25s 664ms/step - loss: 0.5883 - binary_accuracy: 0.6983 - val_loss: 0.5895 - val_binary_accuracy: 0.6863
Epoch 7/100
37/37 [==============================] -

KeyboardInterrupt: 

In [ ]:
data_dictionary["b16"] = [b16_val_x, b16_val_y]

file_path = "b16_history.pickle"

with open(file_path, 'wb') as file_pi:
    pickle.dump(entire_b16_history, file_pi)

In [ ]:
b16_model.save('b16_model.h5')

In [ ]:
b32_history, b32_model, b32_val_x, b32_val_y, entire_b32_history = cross_validate(create_vit_b32, X, Y, kf, datagen, batch_size=batch_size, epochs=epochs)

In [ ]:
data_dictionary["b32"] = [b32_val_x, b32_val_y]

file_path = "b32_history.pickle"

with open(file_path, 'wb') as file_pi:
    pickle.dump(entire_b32_history, file_pi)

In [ ]:
b32_model.save('b32_model.h5')

In [ ]:
with open("data_dictionary.pickle", 'wb') as file_pi:
    pickle.dump(data_dictionary, file_pi)